# SGPS BioStatistics
## Exercise 2a: Dealing with uncertainty from raw experimental data
### Comparing gene expression of TNF, for different doses of LPS, for CRISPR-edited, cell lines.

#### Background on the data
+ Ankylosing Spondylitis is a chronic inflammatory disorder.
+ Through GWAS, the gene *TNFRSF1A* has been implicated in this disease.
+ Would like to understand the regulatory elements around this gene.
+ Using CRISPR/Cas9 to delete putative enhancers and then measured TNF expression for different doses of LPS using qPCR in the resulting cell lines.

In [ ]:
# Import the packages we need
# Manipulate the data
import pandas as pd
import numpy as np
import scipy as sp
# Plot the data
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
# Get the data in
rep1 = pd.read_csv('TNF Dose response Rep 1 summary.csv', index_col=[0,1])
rep1

In [ ]:
# Let's find the mean between the two CQ
rep1['TNF_mean'] = rep1[['TNF Cq 1', 'TNF Cq 2']].mean(axis=1)
rep1['b-actin_mean'] = rep1[['b-actin Cq 1', 'b-actin Cq 2']].mean(axis=1)
rep1

In [ ]:
# Take away the house keeper gene's expression
rep1['DCT'] = rep1['TNF_mean'] - rep1['b-actin_mean']
rep1

In [ ]:
# Take away the unstimulated cell lines' expression
ddct = []
unstimulated_value = 0
for i in rep1.index.values:
    if 'unstimulated' in i:
        unstimulated_value = rep1['DCT'].loc[i]
    ddct.append(rep1['DCT'].loc[i] - unstimulated_value)
rep1['DDCT'] = ddct
rep1['ExDDCT_rep1'] = 2**-rep1['DDCT']
rep1

In [ ]:
# Repeat for 2nd replicate
rep2 = pd.read_csv('TNF Dose response Rep 2 summary.csv', index_col=[0,1])
rep2['TNF_average'] = rep2[['TNF Cq 1', 'TNF Cq 2']].mean(axis=1)
rep2['b-actin_average'] = rep2[['b-actin Cq 1', 'b-actin Cq 2']].mean(axis=1)
rep2['DCT'] = rep2['TNF_average'] - rep2['b-actin_average']
ddct = []
unstimulated_value = 0
for i in rep2.index.values:
    if 'unstimulated' in i:
        unstimulated_value = rep2['DCT'].loc[i]
    ddct.append(rep2['DCT'].loc[i] - unstimulated_value)
rep2['DDCT'] = ddct
rep2['ExDDCT_rep2'] = 2**-rep2['DDCT']
rep2

In [ ]:
# Repeat 3rd replicate
rep3 = pd.read_csv('TNF Dose response Rep 3 summary.csv', index_col=[0,1])
rep3['TNF_average'] = rep3[['TNF Cq 1', 'TNF Cq 2']].mean(axis=1)
rep3['b-actin_average'] = rep3[['b-actin Cq 1', 'b-actin Cq 2']].mean(axis=1)
rep3['DCT'] = rep3['TNF_average'] - rep3['b-actin_average']
ddct = []
unstimulated_value = 0
for i in rep3.index.values:
    if 'unstimulated' in i:
        unstimulated_value = rep3['DCT'].loc[i]
    ddct.append(rep3['DCT'].loc[i] - unstimulated_value)
rep3['DDCT'] = ddct
rep3['ExDDCT_rep3'] = 2**-rep3['DDCT']
rep3

In [ ]:
# Lets collect the data together
all_reps = pd.concat([rep1['ExDDCT_rep1'],rep2['ExDDCT_rep2'],rep3['ExDDCT_rep3']],axis=1)
all_reps

### We want to plot the data, however we also want to add in error bars.
Read the following paper, to get a good idea of what we mean by "error bars".
+  Krzywinski, M & Altman, N, Error bars. Nat Methods 10, 921–922 (2013): https://www.nature.com/articles/nmeth.2659

In [ ]:
# Calculate what we need for error bars
all_reps['Mean'] = all_reps[['ExDDCT_rep1', 'ExDDCT_rep2', 'ExDDCT_rep3']].mean(axis=1)
all_reps['StDev'] = all_reps[['ExDDCT_rep1', 'ExDDCT_rep2', 'ExDDCT_rep3']].std(axis=1)
all_reps['StErr'] = all_reps['StDev'] / np.sqrt(3)

# Calculate the t-critical value for 95% confidence level and 2 degrees of freedom
t_critical = sp.stats.t.ppf(0.975, df=2)  # 0.975 corresponds to a two-tailed 95% confidence interval
print(t_critical)

# Use the calculated t-critical value to compute the confidence interval
all_reps['CI_95'] = all_reps['StErr'] * t_critical

all_reps

In [ ]:
# Lets plot them!
fig, ax = plt.subplots()
cell_lines = {'Intron enh C5':'aqua', 
              'Intergenic enh C8.4':'orange', 
              'Intergenic enh G1':'springgreen', 
              'SFC840-03-03 Ctrl':'hotpink'}

w = 0.2
ind = np.arange(len(all_reps.loc['Intron enh C5'].index))
for c,cell_line in enumerate(cell_lines):
    ax.bar(ind + ((c-1.33)*w), 
            all_reps['Mean'].loc[cell_line].values,
            width = w,
            yerr=all_reps['StErr'].loc[cell_line].values,
            align='center',
            color=cell_lines[cell_line], 
            label=cell_line)


plt.xticks(ind, all_reps.loc['Intron enh C5'].index.values)
plt.legend(loc=2)

In [ ]:
# How about statitical test to see if there are differences within one of the cell lines?
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Filter data for "Intron enh C5"
intron_enh_c5 = all_reps.loc['Intron enh C5']

# Melt the dataframe for One-Way ANOVA
intron_enh_c5_melt = intron_enh_c5.reset_index().melt(id_vars=['dose'], value_vars=['ExDDCT_rep1', 'ExDDCT_rep2', 'ExDDCT_rep3'],
                                                      var_name='Replicate', value_name='Expression')

# Perform One-Way ANOVA
model = ols('Expression ~ C(dose)', data=intron_enh_c5_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("One-Way ANOVA results for 'Intron enh C5':\n", anova_table)

In [ ]:
# Can we tell which groups are significantly different - using Tukey's honest significance test
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Perform Tukey's HSD test
tukey = pairwise_tukeyhsd(endog=intron_enh_c5_melt['Expression'], groups=intron_enh_c5_melt['dose'], alpha=0.05)

# Print the results
print(tukey)

In [ ]:
# Melt the dataframe for Two-Way ANOVA
all_reps_melt = all_reps.reset_index().melt(id_vars=['cell_line', 'dose'], value_vars=['ExDDCT_rep1', 'ExDDCT_rep2', 'ExDDCT_rep3'],
                                            var_name='Replicate', value_name='Expression')

all_reps_melt

In [ ]:
# Perform Two-Way ANOVA
model = ols('Expression ~ C(cell_line) + C(dose) + C(cell_line):C(dose)', data=all_reps_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("Two-Way ANOVA results:\n", anova_table)